In [32]:
import cv2
import numpy as np

video_path = 'Train_Fall_2.mp4'
output_path = 'output_with_boxes.mp4'

cap = cv2.VideoCapture(video_path)

# Resize dimensions (optional)
resize_width = 720
resize_height = 1280

# Video writer settings
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (resize_width, resize_height))

prev_frame = None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize to match writer dimensions
    frame = cv2.resize(frame, (resize_width, resize_height))
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    if prev_frame is None:
        prev_frame = gray
        continue

    diff = cv2.absdiff(prev_frame, gray)
    _, thresh = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)
    thresh = cv2.dilate(thresh, None, iterations=2)

    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        if cv2.contourArea(cnt) > 500:
            x, y, w, h = cv2.boundingRect(cnt)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(frame, "Motion Detected", (x, y-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    out.write(frame)
    prev_frame = gray

cap.release()
out.release()
cv2.destroyAllWindows()


In [23]:
import cv2
import numpy as np

# Input and output
video_path = 'Train_Run_4.mp4'
output_path = 'output_running_only.mp4'

cap = cv2.VideoCapture(video_path)

resize_dim = (640, 360)
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, resize_dim)

prev_frame = None
prev_centroids = []

frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, resize_dim)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    if prev_frame is None:
        prev_frame = gray
        frame_count += 1
        continue

    # Motion detection
    diff = cv2.absdiff(prev_frame, gray)
    _, thresh = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)
    thresh = cv2.dilate(thresh, None, iterations=2)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    current_centroids = []

    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 1000:  # filter small areas
            x, y, w, h = cv2.boundingRect(cnt)
            cx, cy = x + w // 2, y + h // 2
            current_centroids.append((cx, cy))

            # Check speed by comparing with previous frame centroids
            for pcx, pcy in prev_centroids:
                distance = np.sqrt((cx - pcx)**2 + (cy - pcy)**2)
                speed = distance * fps  # pixels per second

                if speed > 1:  # running threshold (tune if needed)
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
                    cv2.putText(frame, f"Running", (x, y - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    prev_centroids = current_centroids
    prev_frame = gray.copy()
    out.write(frame)

    # Optional for debug
    # cv2.imshow("Running Detection", frame)
    # if cv2.waitKey(1) == ord('q'):
    #     break

    frame_count += 1

cap.release()
out.release()
cv2.destroyAllWindows()
